<a href="https://colab.research.google.com/github/MHoseinHoushmand/Clustering_by_SLFA/blob/main/Clustering_by_SLFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pdb
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import norm
import operator
import random

In [2]:
categories = [  #Select 4 categories from fetch_20newsgroups dataset
    "alt.atheism",
    "comp.graphics",
    "sci.space",
    "rec.sport.hockey",
]

dataset = fetch_20newsgroups( #Preprocessing before using dataset
    remove=("headers", "footers", "quotes"),
    subset="all",
    categories=categories,
    shuffle=True,
    random_state=42,
)
labels = dataset.target
unique_labels, category_sizes = np.unique(labels, return_counts=True)
true_k = unique_labels.shape[0]
print(f"{len(dataset.data)} documents - {true_k} categories")

3758 documents - 4 categories


In [3]:
# Vectorize all document as their term frequency(tfidf score)
def docs_as_tfidf(docs):
  vectorizer = TfidfVectorizer(
     max_df=0.5, #Removing terms that are used in more than 50% of articles
     min_df=10,   #Removing terms that are not used in less than 10 of articles
     stop_words="english",
     #  max_features=1000,
  )
  docs_vector = vectorizer.fit_transform(docs)
  return docs_vector.toarray()

In [5]:
population_size = 120 # Frogs number
memplex_num = 12 #define as m
memplex_size = 10 #define as n
max_iteration = 60 #Total Iteration
memplex_iteration = 8 #Iteration As local search
docs = dataset.data
docs_vector = docs_as_tfidf(docs)
print(list(docs_vector[0]))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [6]:
def cosin_sim(a,b):
   return cosine_similarity([a], [b])[0][0]

In [7]:
#Calculate sum of squared error(SSE) as similarity of each documents with the cluster mean in document
def SSE(cluster,doc_mean):
  size = len(cluster)
  sse=0
  for doc in cluster:
    sse += cosin_sim(doc,doc_mean)**2
  sse = sse/size
  return sse

In [ ]:
#Calculate similarity between clusters
def BC(doc_means):
   BC=0
   size = len(doc_means)
   for i in range(size):
      for j in range(i+1,size):
          BC += cosin_sim(doc_means[i],doc_means[j])**2
   return BC

In [9]:
#Calculate similarity within clusters
def WC(clusters):
    WC = 0
    for cluster in clusters:
        doc_mean = np.average(cluster, axis=0)
        WC += SSE(cluster,doc_mean)
    return WC

In [10]:
def build_clusters(answer,docs_vector,clusters_size):
   clusters = []
   for i in range(clusters_size):
       clusters.append([])
   for j in range(len(answer)):
       if -1 < answer[j]:
        clusters[answer[j]].append(docs_vector[j])
   return clusters

In [11]:
def fitness(answer,docs_vector,clusters_size):
   doc_means = []
 #  pdb.runcall(build_clusters,answer,docs_vector,clusters_size)
   clusters = build_clusters(answer,docs_vector,clusters_size)
   for i in range(clusters_size):
       doc_means.append(
          np.average(clusters[i], axis=0)
       )
   wc = WC(clusters)
   bc = BC(doc_means)
   fitness = wc/bc
   return fitness

In [12]:
def cross_over(answer_a,answer_b):
    size = len(answer_a)
    output = []
    for i in range(size):
       choice = random.choice([0,1])
       if choice == 0:
          output.append(answer_a[i])
       else:
          output.append(answer_b[i])
    return tuple(output)

In [13]:
def best_and_worst(answers):
     best =  max(answers, key=answers.get)
     worst = min(answers, key=answers.get)
     return tuple(best) , tuple(worst)


In [14]:
def global_best(memplexes):
     local_bests = {}
     for memplex in memplexes:
         local_best =  max(memplex, key=memplex.get)
         local_bests[local_best]= memplex[local_best]
     global_best = max(local_bests, key=local_bests.get)
     return global_best, local_bests[global_best]


In [15]:
def keys_to_remove(keys , dict):
   for k in keys:
      if k in dict:
          dict.pop(k)
   return dict

In [16]:
def mutation(global_best,clusters_size):
    new_ans = list(global_best)
    size = int(len(global_best)/4)
    indexes = np.random.choice(np.arange(0,len(global_best)), size=size, replace=False)
    values= [random.randint(0, 3) for _ in range(size)]
    for i in range(size):
      new_ans[indexes[i]] = values[i]
    return tuple(new_ans)

In [17]:
def Create_memplexes(population,memplex_num):
     population = dict( sorted(population.items(), key=operator.itemgetter(1), reverse=True))
     memplexes = []
     keys = list(population.keys())
     population_size = len(population)
     for i in range(memplex_num):
         memplexes.append({})
     for i in range(population_size):
         memplexes[i % memplex_num][keys[i]] = population[keys[i]]
     return memplexes

In [18]:
def create_submemplex(memplex,memplex_size, submemplex_size):
    sub_memplex = {}
    prob_list = []
    keys = []
    for i in range(memplex_size):
       for j in range(2*(memplex_size-i)):
          prob_list.append(i)
    k=0
    while(k!=submemplex_size):
       index = random.choice(prob_list)
       key = list(memplex.keys())[index]
       if key not in keys:
           sub_memplex[key] = memplex[key]
           keys.append(key)
           k+=1
    return sub_memplex

In [19]:
def shufeling(memplexes):
    output = {}
    for memplex in memplexes:
        output.update(memplex)
    return output

In [20]:
import operator
def frog_leaping_search(docs_vector,cluster_size):
             answers=np.random.randint(0, cluster_size, size=(population_size , len(dataset.data)))
             population = {}
             i=0
             for answer in answers:
               # pdb.runcall(fitness,answer,docs_vector,cluster_size)
                i+=1
                population[tuple(answer)] = fitness(answer,docs_vector,cluster_size)
                print(i,population[tuple(answer)])

             for i in range(max_iteration):
                #population = dict( sorted(population.items(), key=operator.itemgetter(1), reverse=True))
           #    pdb.runcall(Create_memplexes,population, memplex_num)
                memplexes = Create_memplexes(population, memplex_num)
                population.clear()
              #  pdb.set_trace()
                for j in range(memplex_num):
                    print(i,j,len(memplexes[j]))
                  #  pdb.runcall(create_submemplex,memplexes[j],memplex_size, 5)
                    sub_memplex = create_submemplex(memplexes[j],memplex_size, 5)
                    memplexes[j] =  keys_to_remove(sub_memplex.keys(),memplexes[j])
                    for k in range(memplex_iteration):
                        #pdb.runcall(best_and_worst,sub_memplex)

                   #     for m in sub_memplex:\n",
                    #        print(list(m))
                        ans_best, ans_worst = best_and_worst(sub_memplex)
                        ans_out = cross_over(ans_best,ans_worst)
                        fitness_out = fitness(ans_out,docs_vector,cluster_size)
                        #sec B,
                        ###############################################
                        if len(sub_memplex)< 5:
                               print("errrrrrrrrrrrorrrrrrrrrrrrrB")
                               pdb.set_trace()
                      ###############################################
                      #pdb.runcall(best_and_worst,sub_memplex)
                       # print("############")


                        if (sub_memplex[ans_worst]<fitness_out):
                            del sub_memplex[ans_worst]
                            sub_memplex[ans_out] = fitness_out
                            #sec C
                           ###############################################
                            if len(sub_memplex)< 5:
                                      print("errrrrrrrrrrrorrrrrrrrrrrrrC")
                                      pdb.set_trace()
                          ###############################################
                        else:
                           # pdb.runcall(global_best,memplexes)
                            g_best, g_value = global_best(memplexes)
                            ans_out = cross_over(g_best,ans_worst)
                            #sec D
                            ###############################################
                            if len(sub_memplex)< 5:
                                  print("errrrrrrrrrrrorrrrrrrrrrrrrD")
                                  pdb.set_trace()
                            ###############################################
                            fitness_out = fitness(ans_out,docs_vector,cluster_size)
                            if (sub_memplex[ans_worst] < fitness_out):
                                del sub_memplex[ans_worst]
                                sub_memplex[ans_out] = fitness_out
                                #sec E
                                ###############################################
                                if len(sub_memplex)< 5:
                                      print("errrrrrrrrrrrorrrrrrrrrrrrrE")
                                      pdb.set_trace()
                            ###############################################
                            else:
                           #     print(\"#########\")
                            #    for m in sub_memplex:
                             #        print(list(m))
                                del sub_memplex[ans_worst]
                             #   pdb.runcall(mutation,g_best,cluster_size)
                                ans_out = mutation(g_best,cluster_size)
                                fitness_out = fitness(ans_out,docs_vector,cluster_size)
                                sub_memplex[ans_out] = fitness_out
                                #sec F
                                ###############################################
                                if len(sub_memplex)< 5:
                                      print("errrrrrrrrrrrorrrrrrrrrrrrrF")
                                      pdb.set_trace()
                               ###############################################
                #     pdb.runcall(join_dicts,memplexes[j],sub_memplex)
                    memplexes[j].update(sub_memplex)
                g_best, g_value = global_best(memplexes)
               # pdb.runcall(show_result,g_best)
                print(g_best)
                print(g_value)
                population = shufeling(memplexes)
             return g_best, g_value, population

In [21]:
g_best, g_value,population = frog_leaping_search(docs_vector,4)
true = 0
size = len(dataset.data)
print(labels)
print(list(g_best))

[1, 3, 1, 0, 1, 3, 0, 1, 0, 3, 2, 3, 2, 2, 1, 3, 1, 0, 1, 2, 2, 2, 0, 0, 2, 3, 2, 3, 3, 2, 1, 0, 0, 2, 3, 3, 1, 1, 1, 3, 3, 2, 1, 0, 2, 3, 2, 2, 3, 0, 0, 2, 2, 0, 3, 0, 2, 1, 0, 0, 0, 3, 2, 0, 2, 3, 0, 0, 3, 0, 3, 2, 2, 0, 1, 3, 2, 1, 2, 1, 3, 0, 1, 1, 2, 0, 1, 2, 0, 0, 0, 0, 3, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2, 3, 0, 2, 1, 0, 2, 1, 0, 2, 0, 1, 2, 2, 3, 3, 1, 3, 0, 0, 1, 1, 3, 2, 0, 2, 1, 0, 2, 1, 3, 2, 3, 1, 1, 2, 2, 2, 0, 3, 3, 2, 3, 2, 1, 0, 1, 1, 2, 2, 2, 1, 1, 3, 3, 0, 2, 2, 3, 0, 0, 2, 1, 3, 3, 2, 1, 2, 0, 0, 1, 1, 2, 1, 1, 2, 0, 3, 1, 0, 2, 0, 1, 3, 1, 1, 3, 2, 2, 2, 0, 1, 0, 2, 1, 3, 3, 0, 1, 2, 0, 3, 2, 3, 3, 3, 2, 1, 2, 3, 3, 0, 2, 1, 0, 2, 3, 1, 0, 0, 0, 2, 2, 1, 0, 0, 2, 2, 0, 3, 0, 0, 3, 3, 0, 2, 3, 0, 0, 2, 2, 1, 3, 1, 2, 3, 0, 2, 3, 0, 2, 0, 3, 0, 2, 3, 1, 1, 2, 0, 1, 2, 0, 0, 3, 3, 1, 2, 1, 3, 3, 3, 2, 1, 0, 0, 0, 3, 0, 2, 0, 3, 2, 3, 1, 1, 1, 0, 0, 1, 0, 3, 2, 2, 2, 1, 2, 2, 1, 2, 3, 0, 3, 3, 3, 0, 2, 1, 3, 1, 0, 2, 3, 1, 1, 2, 0, 0, 1, 0, 3, 2, 3, 3, 3, 3, 2, 2, 0, 0, 3, 

KeyboardInterrupt: ignored